In [55]:
%matplotlib inline

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial
import dgl

In [57]:
# load graph data
from dgl.contrib.data import load_data
import numpy as np
data = load_data('FB15k-237')
num_nodes = data.num_nodes
num_rels = data.num_rels

# entities: 14541
# relations: 237
# edges: 272115


In [58]:
len(data.test)

20466

In [59]:
import pandas as pd

In [60]:
data=pd.DataFrame(data.test, columns=['source','edge','target'])

In [61]:
edge = data['edge'].value_counts()[:8].index
edge = pd.DataFrame(edge, columns=['edge'])
edge['relation'] = ['INTERESTED_IN','BOOKMARK','TALK_OF','ASSUME_UNDERSTANDING_OF','WRITTEN_BY','ENCAPSULATES','SYNONYM','REPOSITORY_OF']

In [62]:
edge_dict = dict(zip(edge.edge,edge.relation))

In [63]:
data['edge'].value_counts()[:8].index

Int64Index([75, 2, 125, 108, 82, 4, 59, 111], dtype='int64')

In [64]:
data['edge_name'] = data.edge.map(edge_dict)

In [65]:
data = data.dropna()

In [66]:
inter_in = data[data.edge==75].reset_index(drop=True)
inter_in['source_name'] = 'user_'+inter_in['source'].astype(str)
inter_in['target_name'] = 'topic_'+inter_in['target'].astype(str)

In [67]:
bookmark = data[data.edge==2].reset_index(drop=True)
bookmark['source_name'] = 'user_'+bookmark['source'].astype(str)
content = bookmark[bookmark.target<5100]
content['target_name'] = 'content_'+content['target'].astype(str)
extRes = bookmark[bookmark.target>=5100]
extRes['target_name'] = 'externalResources_'+extRes['target'].astype(str)
bookmark = extRes.append(content).reset_index(drop=True)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
talk_of = data[data.edge==125]

In [69]:
content = talk_of[talk_of.source>8000]
content['source_name'] = 'content_'+content['source'].astype(str)
content['target_name'] = 'topic_'+content['target'].astype(str)
thread = talk_of[talk_of.source<=8000]
thread['source_name'] = 'thread_'+thread['source'].astype(str)
thread['target_name'] = 'book_'+thread['target'].astype(str)
talk_of = thread.append(content)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [70]:
auo = data[data.edge==108]
auo['target_name'] = 'topic_'+auo['target'].astype(str)
content = auo[auo.source>7000]
content['source_name'] = 'content_'+content['source'].astype(str)
topic = auo[auo.source<=7000]
topic['source_name'] = 'topic_'+topic['source'].astype(str)
auo = topic.append(content)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [71]:
written_by = data[data.edge==82]
written_by['source_name'] = 'content_'+written_by['source'].astype(str)
written_by['target_name'] = 'user_'+written_by['target'].astype(str)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
encaps = data[data.edge==4]
encaps['source_name'] = 'library_'+encaps['source'].astype(str)
method = encaps[encaps.target>12000]
method['target_name'] = 'mlStatMethod_'+method.target.astype(str)
algo = encaps[(encaps.target>4500)&(encaps.target<=12000)]
algo['target_name'] = 'mlStatAlgo_'+algo.target.astype(str)
metric = encaps[encaps.target<=4500]
metric['target_name'] = 'mlStatMetric_'+metric.target.astype(str)
encaps=metric.append([algo,method])

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [73]:
syn = data[data.edge==59]
syn['source_name'] = 'topic_'+syn.source.astype(str)
syn['target_name'] = 'topic_'+syn.target.astype(str)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
rep_of = data[data.edge==59]
rep_of['source_name'] = 'externalResources_'+syn.source.astype(str)
rep_of['target_name'] = 'library_'+rep_of.target.astype(str)

C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [75]:
data = inter_in.append([bookmark,talk_of,auo,written_by,encaps,syn,rep_of]).reset_index(drop=True)

C:\Aplikasi\Anaconda\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [76]:
data['source_type'] = [a.split('_')[0] for a in data.source_name]
data['target_type'] = [a.split('_')[0] for a in data.target_name]

In [77]:
data

,edge,edge_name,source,source_name,target,target_name,source_type,target_type
0,75,INTERESTED_IN,668,user_668,1499,topic_1499,user,topic
1,75,INTERESTED_IN,10319,user_10319,2414,topic_2414,user,topic
2,75,INTERESTED_IN,31,user_31,3056,topic_3056,user,topic
3,75,INTERESTED_IN,2471,user_2471,11898,topic_11898,user,topic
4,75,INTERESTED_IN,13606,user_13606,3056,topic_3056,user,topic
...,...,...,...,...,...,...,...,...
7564,59,SYNONYM,2244,externalResources_2244,5838,library_5838,externalResources,library
7565,59,SYNONYM,1921,externalResources_1921,7664,library_7664,externalResources,library
7566,59,SYNONYM,5436,externalResources_5436,7977,library_7977,externalResources,library
7567,59,SYNONYM,5436,externalResources_5436,13770,library_13770,externalResources,library


In [78]:
for a,b in enumerate(data.values):
    if data.source_type.at[a]=='user':
        data.source.at[a]=0
        data.source_name.at[a]='user_0'
    elif data.target_type.at[a]=='user':
        data.target.at[a]=0
        data.target_name='user_0'

In [97]:
type_node = set(np.concatenate([data.source_name.unique(),data.target_name.unique()]))
type_node = sorted(type_node)

In [98]:
type_node=pd.DataFrame(type_node,columns=['nodes'])
type_node = dict(type_node.nodes)
type_node = {v: k for k, v in type_node.items()}

In [103]:
data.source = data.source_name.map(type_node)
data.target = data.target_name.map(type_node)

In [121]:
type_edge = sorted(data.edge_name.unique())

In [122]:
type_edge=pd.DataFrame(type_edge,columns=['edges'])
type_edge = dict(type_edge.edges)
type_edge = {v: k for k, v in type_edge.items()}
data.edge = data.edge_name.map(type_edge)

In [123]:
data[['source','edge','target','source_name','edge_name','target_name']].to_csv('sample_dataset_learnavi.csv')

In [124]:
data

,edge,edge_name,source,source_name,target,target_name,source_type,target_type
0,3,INTERESTED_IN,2511,user_0,2511,user_0,user,topic
1,3,INTERESTED_IN,2511,user_0,2511,user_0,user,topic
2,3,INTERESTED_IN,2511,user_0,2511,user_0,user,topic
3,3,INTERESTED_IN,2511,user_0,2511,user_0,user,topic
4,3,INTERESTED_IN,2511,user_0,2511,user_0,user,topic
...,...,...,...,...,...,...,...,...
7564,4,SYNONYM,1121,externalResources_2244,2511,user_0,externalResources,library
7565,4,SYNONYM,1115,externalResources_1921,2511,user_0,externalResources,library
7566,4,SYNONYM,1169,externalResources_5436,2511,user_0,externalResources,library
7567,4,SYNONYM,1169,externalResources_5436,2511,user_0,externalResources,library


In [120]:
data.edge_name.map(type_edge)

0       3
1       3
2       3
3       3
4       3
       ..
7564    4
7565    4
7566    4
7567    4
7568    4
Name: edge_name, Length: 7569, dtype: int64